# **Evaluate**

    Synthetic Dataset Form을 가지고 진행해야 함.
    - Query
    - Corpus
    - Relavant Docs

In [1]:
# !pip install llama_index
# !pip install sentence_transformers

  Using cached aiohttp-3.9.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (7.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 943.4/943.4 kB 3.6 MB/s eta 0:00:00m eta 0:00:010:00:01
Using cached aiohttp-3.9.1-cp310-cp310-macosx_11_0_arm64.whl (386 kB)
  Attempting uninstall: nest-asyncio
    Found existing installation: nest-asyncio 1.5.6
    Uninstalling nest-asyncio-1.5.6:
      Successfully uninstalled nest-asyncio-1.5.6
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.8.5
    Uninstalling aiohttp-3.8.5:
      Successfully uninstalled aiohttp-3.8.5


In [2]:
import json
from tqdm.notebook import tqdm
import pandas as pd

from llama_index import ServiceContext, VectorStoreIndex
from llama_index.schema import TextNode
from llama_index.embeddings import OpenAIEmbedding

In [3]:
def evaluate(dataset, embed_model, top_k=5, verbose=False):
    corpus = dataset['corpus']
    queries = dataset['queries']
    relevant_docs = dataset['relevant_docs']

    service_context = ServiceContext.from_defaults(embed_model=embed_model)
    nodes = [TextNode(id_=id_, text=text) for id_, text in corpus.items()]
    index = VectorStoreIndex(
        nodes,
        service_context=service_context,
        show_progress=True
    )
    retriever = index.as_retriever(similarity_top_k=top_k)

    eval_results = []
    for query_id, query in tqdm(queries.items()):
        retrieved_nodes = retriever.retrieve(query)
        retrieved_ids = [node.node.node_id for node in retrieved_nodes]
        expected_id = relevant_docs[query_id][0]
        is_hit = expected_id in retrieved_ids  # assume 1 relevant doc

        eval_result = {
            'is_hit': is_hit,
            'retrieved': retrieved_ids,
            'expected': expected_id,
            'query': query_id,
        }
        eval_results.append(eval_result)
    return eval_results

In [4]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from sentence_transformers import SentenceTransformer

def evaluate_st(dataset, model_id, name):
    corpus = dataset['corpus']
    queries = dataset['queries']
    relevant_docs = dataset['relevant_docs']

    evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs, name=name)
    model = SentenceTransformer(model_id)
    return evaluator(model, output_path='results/')

/Users/choejaehun/anaconda3/envs/ssis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# **Run**

In [ ]:
bge = "local:BM-K/KoSimCSE-roberta-multitask"
### pretrained embedding model config
model = SentenceTransformer('BM-K/KoSimCSE-roberta-multitask')
results = evaluate(corpus, model)

In [ ]:
df_res = pd.DataFrame(results)

In [ ]:
hit_rate_bge = df_res['is_hit'].mean()
hit_rate_bge

In [ ]:
evaluate_st(val_corpus, "BM-K/KoSimCSE-roberta-multitask", name='bge')